# Exploring Text with SpaCy

## The spaCy pipeline

"When you call nlp on a text, spaCy first tokenizes the text to produce a Doc object. The Doc is then processed in several different steps – this is also referred to as the processing [pipeline](https://spacy.io/usage/processing-pipelines)." 

From https://course.spacy.io/en/chapter3

<img src="https://spacy.io/images/pipeline.svg">

In [1]:
import sys
import spacy
#from spacy.tokens import Doc, Span
nlp = spacy.load("en_core_web_md") #_sm does not have word vectors
print(nlp.pipe_names)

['tok2vec', 'tagger', 'parser', 'attribute_ruler', 'lemmatizer', 'ner']


## Adding sentiment via TextBlob

In [2]:
from spacytextblob.spacytextblob import SpacyTextBlob
nlp.add_pipe('spacytextblob')
print(nlp.pipe_names)

['tok2vec', 'tagger', 'parser', 'attribute_ruler', 'lemmatizer', 'ner', 'spacytextblob']


## Exploring document structure

In [3]:
# Break the corpus into:
# Title, Chapters, Sections, Paragraphs, Thoughts, Sentences
#  Title: Knowing Gaia
#  C1:Introduction 1 section, 6 paragraphs
#  C2:Taking a leap 8 sections, 7 paragraphs
#  C3-C10: each with 10 sections  thought-embedding 
#  C11: Contemplation: 2 sections

In [50]:
# Open the text file and read its content into a variable
with open('../assets/raw.txt', 'r') as file:
    text = file.read()

doc=nlp(text)

In [51]:
# Before extra bash cleaning:
#print([tok.text for tok in doc[21:30] 
# if not tok.is_stop and tok.text!="\n\n"])

# After extra bash cleaning:
print([tok.text for tok in doc[21:30]])

['traditional', 'non', '-', 'duality', 'speakers', ',', 'I', 'was', 'inspired']


In [23]:
sents=[ sent for sent in doc.sents]
type(sents[0])

spacy.tokens.span.Span

In [24]:
nlp.vocab["\n"].is_stop = True
words=[token.text for token in doc
       if not token.is_stop and not token.is_punct ]

In [25]:
from collections import Counter
word_freq=Counter(words)

In [26]:
common_words = word_freq.most_common(30)

In [27]:
print(common_words)

[('Gaia', 190), ('\n\n', 170), ('want', 99), ('know', 78), ('people', 75), ('self', 71), ('like', 68), ('system', 61), ('need', 58), ('think', 53), ('experience', 52), ('feel', 48), ('life', 46), ('feeling', 46), ('power', 44), ('body', 41), ('dream', 40), ('attention', 39), ('person', 38), ('mind', 35), ('systems', 34), ('living', 33), ('right', 33), ('way', 30), ('ideology', 30), ('emotions', 29), ('work', 29), ('time', 28), ('thinking', 27), ('away', 26)]


In [28]:
for token in doc[1:10]:
    print(token, token.tag_, token.pos_, spacy.explain(token.tag_))

Gaia NNP PROPN noun, proper singular


 _SP SPACE whitespace
C1 NN NOUN noun, singular or mass
: : PUNCT punctuation mark, colon or ellipsis
Introduction NN NOUN noun, singular or mass

 _SP SPACE whitespace
After IN ADP conjunction, subordinating or preposition
many JJ ADJ adjective (English), other noun-modifier (Chinese)
years NNS NOUN noun, plural


In [29]:
nouns = []
adjectives = []
for token in [t for t in doc[2:4000] if t.text != "-"]:
    if token.pos_ == 'NOUN':
        nouns.append(token)
    if token.pos_ == 'ADJ':
        adjectives.append(token)

print(f'Total nouns: {len(nouns)} \nFirst 5 nouns: {nouns[:5]}')
print(f'Total adjectives: {len(adjectives)}')
print(f'First 5 adjectives: {adjectives[:5]}')

Total nouns: 677 
First 5 nouns: [C1, Introduction, years, study, practice]
Total adjectives: 281
First 5 adjectives: [many, Buddhist, less, traditional, non]


In [30]:
from spacy import displacy
# Creates large image of relations
#displacy.render(ss[3:4], style='dep', jupyter=True)

### A SpaCy Document is a list of tokens
SpaCy **Doc** data structure is a sequence of Token objects allowing access to **sentences** 
and named entities.

In [31]:
# Get all tokens and part-of-speech tags
token_texts = [token.text for token in doc]
pos_tags = [token.pos_ for token in doc]

# Iterate over first 2000 tokens
for token in doc[1:2000]:
    # Check if the current token is a proper noun
    if token.pos_ == "PROPN":
        # Check if the next token is a verb
        if doc[token.i + 1].pos_ == "VERB":
            print("Found proper noun before a verb:", token.text)

center=4
width=4
print(f'{len(doc[1].vector)}')
print(f'Simularity between {doc[center].text} and \
        {doc[center].similarity(doc[center-width:center+width])}')

Found proper noun before a verb: NASA
Found proper noun before a verb: LeGuin
Found proper noun before a verb: Gaia
300
Simularity between : and         0.4422609806060791


In [32]:
# Each vocabulary word has a 300 dim vector which is a word2vec vector
# Dot product is not normalized but seems to work

In [33]:
import numpy
numpy.dot(doc.vocab['earth'].vector,doc.vocab['gaia'].vector)

0.8249819

In [34]:
from itertools import product
from itertools import zip_longest
c=["C1", "C2", "C3", "C4", "C5", "C6", "C7", "C8", "C9", "C10", "C11"]
starts=[ t.i for chap in c for t in doc if t.text==chap]
ci=list(zip_longest(starts,starts[1:]))
chaps=[ doc[s[0]:s[1]] for s in ci] # list of spans
print(ci)

[(3, 720), (720, 1912), (1912, 3603), (3603, 4985), (4985, 7204), (7204, 9617), (9617, 12594), (12594, 16739), (16739, 20890), (20890, 24154), (24154, None)]


In [35]:
pts=["P1", "P2", "P3", "P4", "P5", "P6", "P7", "P8", "P9", "P10", "P11"]
chap=5
prompt=3
pis=[ t.i for pstr in pts for t in chaps[chap] if t.text==pstr]
pi=list(zip_longest(pis,pis[1:])) # iterator can only be used once
ptexts=[ chaps[chap][s[0]:s[1]] for s in pi]
chap6prompts=[ doc[s[0]:s[1]] for s in pi]
sents=list(doc[pis[prompt-1]:pis[prompt]].sents)

In [36]:
with open("./eda.html", 'w', encoding='utf-8') as f:
    f.write("<!DOCMENT HTML>")
    f.write("<html><head><style>")
    f.write("html{ font-family: monospace; color:#888}")    
    f.write("</style>")
    for c in chaps:
        f.write(f'<h1>{c[1:3].text}</h1>')
        f.write(c.text)
        
    f.write("</html>")

## Adding spans to Doc

In [37]:
from spacy import displacy
from spacy.tokens import Span

sents = list(chaps[0].sents)
type(doc)
type(sents)
options = {"colors": {"PHRASE1": "green",
                      "PHRASE2": "red",
                      "THOUGHT": "orange"}}

# type(chap[0]) = spacy.tokens.span.Span
# type(chap[0][1:60]) = spacy.tokens.span.Span

smalldoc=chaps[0][0:64].as_doc()
smalldoc.spans["sc"] = [
 
    Span(doc, 24, 36, "PHRASE2"),
    Span(doc, 3, 35, "THOUGHT"),
      Span(doc, 4, 23, "PHRASE1"), 

]
displacy.render( smalldoc, style='span',options=options)

## Create the document structure
We have 11 Chapters

In [66]:
import spacy

# Load the SpaCy model for English
nlp = spacy.load("en_core_web_sm")

def is_int(s):
    try: 
        int(s)
        return True
    except ValueError:
        return False

def parse_document(text):
    chapters_list = []
    chapter_dict = None
    prompt_list = []
    paragraph_list = []
    
    # Split the text into lines
    lines = text.split('\n')

    for line in lines:
        line = line.strip()  # Remove leading/trailing white spaces
        if not line:
            continue  # Skip empty lines
        
        if line.startswith('C') and ':' in line:
            # Save previous chapter if exists
            if chapter_dict is not None:
                chapter_dict["prompts"] = prompt_list
                chapters_list.append(chapter_dict)
            
            parts = line.split(':', 1)
            chapter_number = parts[0][1:].strip()
            chapter_title = parts[1].strip()
            chapter_dict = {
                "chapter_number": int(chapter_number) if is_int(chapter_number) else None,
                "chapter_title": chapter_title
            }
            prompt_list = []  # Reset prompts
            continue
        
        if line.startswith('P') and ':' in line:
            # Save previous prompt if exists
            if prompt_list and paragraph_list:
                last_prompt = prompt_list[-1]
                last_prompt["paragraphs"] = paragraph_list
                paragraph_list = []  # Reset paragraphs for the new prompt
            
            parts = line.split(':', 1)
            prompt_number = parts[0][1:].strip()
            prompt_keyword = parts[1].strip()
            prompt_list.append({
                "prompt_number": int(prompt_number) if is_int(prompt_number) else None,
                "prompt_keyword": prompt_keyword,
                "paragraphs": []
            })
            continue
        
        # Process sentences in the paragraph
        doc = nlp(line)
        sentences = [sent.text for sent in doc.sents]
        paragraph_list.append({
            "paragraph_number": len(paragraph_list) + 1,
            "sentences": sentences
        })
    
    # Add last prompt and chapter
    if prompt_list and paragraph_list:
        last_prompt = prompt_list[-1]
        last_prompt["paragraphs"] = paragraph_list
    if chapter_dict is not None:
        chapter_dict["prompts"] = prompt_list
        chapters_list.append(chapter_dict)
    
    return {"chapters": chapters_list}


In [67]:
p=parse_document(text)

In [73]:
import json
import subprocess

# Your Python dictionary
data = {"key": "value"}

# Convert the dictionary to a JSON-formatted string
json_string = json.dumps(p)

# Call jq with the subprocess module
result = subprocess.run(["jq", '.'], input=json_string, text=True, capture_output=True)

# Output from jq
print(result.stdout)

{
  "chapters": [
    {
      "chapter_number": 1,
      "chapter_title": "Introduction",
      "prompts": []
    },
    {
      "chapter_number": 2,
      "chapter_title": "Taking a leap into the deeply personal",
      "prompts": [
        {
          "prompt_number": 1,
          "prompt_keyword": "Since living systems do not exist independently in nature, what am I? What does that feel like?",
          "paragraphs": [
            {
              "paragraph_number": 1,
              "sentences": [
                "Knowing Gaia"
              ]
            },
            {
              "paragraph_number": 2,
              "sentences": [
                "After many years of Buddhist study and practice, and being influenced by less traditional non-duality speakers, I was inspired to develop a contemplation practice called “Knowing Gaia”.",
                "I’ve combined my experience of meditation and a Buddhist world view with a more contemporary approach which is not based on any s

## References

- [RealPython notes on SpaCy](https://realpython.com/natural-language-processing-spacy-python/#preprocessing-functions)